In [1]:
import os
from dotenv import load_dotenv
import numpy as np
from tqdm.notebook import tqdm
import csv

from huggingface_hub import login
from litellm import completion
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.optim.lr_scheduler import CosineAnnealingLR

from pricer.evaluator import evaluate
from pricer.items import Item



In [2]:
LITE_MODE = False

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 800,000 training items, 10,000 validation items, 10,000 test items


In [4]:
human_predictions = []
with open("human_in.csv", "w", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    for t in test[:100]:
        writer.writerow([t.summary, 0]) 

In [5]:
human_predictions = []
with open("human_out.csv", "r", encoding="utf-8") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        human_predictions.append(float(row[1]))

In [6]:
def human_pricer(item):
    idx = test.index(item)
    return human_predictions[idx]

In [7]:
human = human_pricer(test[0])
actual = test[0].price
print(f"Human predicted {human} for an item that actually cost: {actual}")

Human predicted 120.0 for an item that actually cost: 219.0


In [8]:
evaluate(human_pricer, test, size=100)

  0%|          | 0/100 [00:00<?, ?it/s]

$99 $184 $12 $15 $18 $10 $119 $135 $6 $270 $643 $329 $15 $26 $24 $18 $29 $25 $25 $53 $35 $126 $25 $127 $273 $398 $55 $6 $101 $51 $30 $5 $35 $9 $10 $419 $25 $11 $186 $33 $161 $51 $23 $155 $150 $4 $31 $18 $115 $82 $25 $111 $410 $75 $67 $34 $8 $10 $122 $28 $116 $17 $19 $60 $599 $60 $160 $355 $75 $34 $17 $2 $70 $76 $41 $9 $226 $5 $5 $4 $0 $7 $5 $74 $7 $10 $68 $74 $5 $3 $17 $45 $5 $16 $0 $153 $2 $122 $150 $355 

In [9]:
y = np.array([float(item.price) for item in train])
documents = [item.summary for item in train]

In [10]:
# HashingVectorizer for Bag of Words model, one-hot vector only - don't count occurences of the word
np.random.seed(42)
vectorizer = HashingVectorizer(n_features=5000, stop_words="english", binary=True)
X = vectorizer.fit_transform(documents)

In [11]:
# 8 layer neural network
class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 128)
        self.layer2 = nn.Linear(128, 64)
        self.layer3 = nn.Linear(64, 64)
        self.layer4 = nn.Linear(64, 64)
        self.layer5 = nn.Linear(64, 64)
        self.layer6 = nn.Linear(64, 64)
        self.layer7 = nn.Linear(64, 64)
        self.layer8 = nn.Linear(64, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        output1 = self.relu(self.layer1(x))
        output2 = self.relu(self.layer2(output1))
        output3 = self.relu(self.layer3(output2))
        output4 = self.relu(self.layer4(output3))
        output5 = self.relu(self.layer5(output4))
        output6 = self.relu(self.layer6(output5))
        output7 = self.relu(self.layer7(output6))
        output8 = self.layer8(output7)
        return output8

In [12]:
# Convert data to Pytorch tensors
X_train_tensor = torch.FloatTensor(X.toarray())
y_train_tensor = torch.FloatTensor(y).unsqueeze(1)

# Split into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_tensor, y_train_tensor, test_size=0.01, random_state=42)

# Create the loader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Initialize the model
input_size= X_train_tensor.shape[1]
model = NeuralNetwork(input_size)

In [13]:
trainable_params  = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params:,}")

Number of trainable parameters: 669,249


In [14]:
# Define loss function and optimizer

loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 2 passes

EPOCHS = 2

for epoch in range(EPOCHS):
    model.train()
    for batch_X, batch_y in tqdm(train_loader):
        optimizer.zero_grad()

        # 4 training stages: forward pass, loss calculation, backward pass, optimize
        
        outputs = model(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = loss_function(val_outputs, y_val)

    print(f'Epoch [{epoch+1}/{EPOCHS}], Train Loss: {loss.item():.3f}, Val Loss: {val_loss.item():.3f}')

  0%|          | 0/12375 [00:00<?, ?it/s]

Epoch [1/2], Train Loss: 7344.520, Val Loss: 12344.890


  0%|          | 0/12375 [00:00<?, ?it/s]

Epoch [2/2], Train Loss: 21759.439, Val Loss: 11064.930


In [15]:
def neural_network(item):
    model.eval()
    with torch.no_grad():
        vector = vectorizer.transform([item.summary])
        vector = torch.FloatTensor(vector.toarray())
        result = model(vector)[0].item()
    return max(0, result)

In [16]:
evaluate(neural_network, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$110 $108 $31 $83 $30 $69 $68 $57 $37 $96 $283 $112 $75 $55 $63 $9 $22 $15 $11 $39 $45 $72 $25 $42 $184 $114 $247 $34 $83 $48 $102 $169 $71 $5 $54 $379 $26 $38 $90 $65 $151 $9 $11 $9 $6 $64 $19 $11 $4 $8 $31 $37 $153 $6 $86 $111 $34 $88 $37 $31 $119 $12 $29 $10 $343 $228 $3 $320 $34 $118 $8 $18 $23 $93 $9 $38 $135 $14 $36 $53 $65 $33 $31 $48 $25 $96 $67 $111 $45 $132 $18 $26 $8 $7 $53 $40 $21 $24 $3 $219 $5 $33 $19 $67 $14 $33 $62 $321 $20 $103 $36 $127 $90 $10 $87 $234 $38 $82 $33 $119 $13 $235 $116 $37 $76 $38 $13 $63 $24 $37 $53 $45 $91 $4 $47 $14 $100 $5 $121 $54 $11 $81 $22 $141 $135 $31 $23 $282 $57 $9 $13 $51 $3 $4 $56 $88 $103 $2 $25 $10 $23 $2 $12 $14 $294 $36 $88 $26 $17 $50 $33 $12 $234 $29 $4 $5 $32 $1 $32 $84 $184 $14 $175 $50 $31 $38 $61 $98 $20 $22 $28 $11 $5 $156 $37 $21 $71 $29 $22 $4 

In [19]:
def messages_for(item):
    message = f"Estimate the price of this product.  Respond with the price, no explanation\n\n{item.summary}"
    return [{"role": "user", "content": message}]

In [21]:
print(test[0].summary)

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.


In [22]:
def gpt_4_1_nano(item):
    response = completion(model="openai/gpt-4.1-nano", messages=messages_for(item))
    return response.choices[0].message.content

In [23]:
gpt_4_1_nano(test[0])

'$180'

In [24]:
print(len(test))

10000


In [25]:
test[0].price

219.0

In [26]:
evaluate(gpt_4_1_nano, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$19 $34 $25 $20 $120 $80 $6 $65 $1 $870 $137 $80 $15 $9 $19 $7 $41 $0 $140 $31 $54 $16 $65 $125 $182 $303 $405 $0 $501 $60 $30 $20 $10 $50 $35 $119 $90 $31 $34 $23 $150 $45 $20 $105 $70 $0 $17 $13 $75 $52 $20 $105 $225 $0 $97 $16 $8 $110 $52 $3 $86 $18 $31 $40 $179 $5 $90 $295 $25 $44 $16 $8 $20 $1 $5 $21 $126 $0 $8 $3 $30 $3 $5 $74 $11 $15 $32 $56 $30 $26 $13 $25 $5 $20 $6 $78 $1 $93 $20 $425 $50 $33 $21 $11 $150 $182 $10 $350 $4 $99 $10 $86 $29 $22 $30 $230 $5 $5 $24 $47 $19 $311 $50 $34 $50 $10 $5 $81 $29 $64 $79 $113 $35 $0 $85 $10 $55 $10 $48 $42 $16 $0 $70 $10 $114 $118 $5 $340 $115 $13 $4 $144 $12 $10 $3 $29 $101 $41 $30 $5 $311 $17 $3 $2 $390 $7 $752 $30 $5 $5 $0 $3 $120 $8 $27 $201 $3 $57 $56 $13 $246 $25 $150 $99 $0 $8 $63 $7 $20 $2 $15 $99 $15 $11 $9 $70 $10 $30 $21 $1 

In [30]:
def claude_opus_4_6(item):
    response = completion(model="anthropic/claude-opus-4-6", messages=messages_for(item))
    return response.choices[0].message.content

In [33]:
evaluate(claude_opus_4_6, test, size=20, workers=1)

  0%|          | 0/20 [00:00<?, ?it/s]

$20 $4 $15 $40 $20 $30 $54 $70 $10 $45 $363 $129 $7 $22 $39 $4 $11 $25 $20 $54 

In [41]:
def gpt_5_1(item):
    response = completion(model="gpt-5.1", messages=messages_for(item), reasoning_effort='none', seed=(42))
    return response.choices[0].message.content

In [42]:
evaluate(gpt_5_1, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$10 $104 $5 $60 $10 $140 $54 $85 $6 $90 $113 $30 $10 $9 $39 $7 $11 $20 $80 $1 $76 $14 $45 $15 $112 $204 $65 $1 $111 $64 $12 $20 $20 $55 $10 $169 $60 $34 $24 $8 $160 $35 $15 $65 $50 $5 $1 $1 $75 $58 $24 $107 $185 $0 $27 $6 $6 $80 $128 $5 $86 $48 $36 $50 $120 $10 $80 $325 $20 $84 $14 $2 $60 $0 $25 $17 $46 $2 $2 $7 $10 $4 $5 $74 $2 $25 $68 $86 $10 $11 $8 $10 $5 $10 $1 $78 $1 $77 $80 $290 $30 $33 $7 $20 $49 $102 $15 $340 $1 $69 $20 $206 $14 $8 $54 $30 $6 $1 $24 $417 $14 $9 $0 $24 $10 $5 $5 $31 $39 $54 $149 $3 $0 $5 $135 $2 $35 $10 $22 $8 $11 $49 $20 $10 $14 $13 $15 $120 $85 $3 $1 $114 $22 $60 $1 $41 $21 $36 $30 $5 $39 $18 $2 $1 $241 $4 $651 $20 $10 $2 $5 $2 $200 $8 $59 $41 $4 $17 $26 $8 $4 $25 $240 $29 $10 $3 $53 $7 $10 $3 $5 $19 $10 $111 $35 $50 $10 $50 $11 $6 

In [45]:
def gemma_270(item):
    response = completion(model="ollama/gemma3:270m", messages=messages_for(item))
    return response.choices[0].message.content

In [46]:
evaluate(gemma_270, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$184 $134 $25 $45 $15 $218 $129 $35 $464 $2170 $363 $343 $5 $29 $921 $766 $271 $0 $140 $231 $216 $1076 $72 $75 $168 $403 $145 $8 $51 $41 $100 $30 $125 $76 $35 $319 $20 $31 $986 $82 $160 $55 $5 $251 $91 $0 $82 $18 $95 $248 $23 $100 $725 $230 $447 $34 $43 $10 $192 $18 $64 $422 $56 $390 $479 $39 $60 $195 $225 $51 $18 $18 $255 $281 $42 $1 $226 $10 $33 $231 $90 $188 $21 $64 $27 $30 $168 $44 $30 $21 $23 $50 $479 $15 $6 $78 $1 $127 $170 $225 $150 $113 $17 $114 $199 $1082 $0 $50 $6 $49 $339 $36 $14 $172 $46 $80 $10 $5 $52 $1097 $19 $11 $230 $184 $150 $1239 $380 $64 $149 $11 $22 $2363 $714 $99 $285 $5 $5 $60 $21 $212 $735 $121 $420 $4 $169 $418 $155 $460 $235 $21 $6 $394 $53 $110 $7 $121 $101 $6 $285 $5 $159 $494 $30 $2 $660 $7 $2 $310 $10 $89 $10 $3 $171 $27 $198 $114 $2 $43 $90 $56 $1746 $25 $900 $1 $46 $8 $366 $32 $20 $13 $15 $299 $15 $860 $80 $270 $1129 $70 $29 $11 

In [47]:
def gpt_oss_20(item):
    response = completion(model="ollama/gpt-oss:20b", messages=messages_for(item))
    return response.choices[0].message.content

In [48]:
evaluate(gpt_oss_20, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$281 $134 $20 $21 $30 $110 $64 $65 $6 $1370 $263 $99 $80 $9 $19 $3 $41 $45 $95 $20 $35 $136 $50 $75 $133 $203 $70 $20 $71 $55 $28 $30 $190 $60 $70 $31 $60 $27 $44 $28 $165 $62 $10 $206 $185 $3 $6 $23 $90 $23 $17 $105 $125 $25 $1747 $99 $13 $280 $58 $6 $44 $21 $259 $40 $599 $39 $85 $235 $55 $204 $10 $18 $70 $1 $22 $2 $326 $5 $13 $1 $10 $6 $19 $74 $13 $0 $32 $26 $40 $11 $3 $40 $0 $35 $2 $8 $4 $72 $205 $75 $80 $23 $2 $44 $139 $532 $10 $355 $19 $401 $80 $406 $149 $22 $4 $780 $7 $2 $224 $52 $9 $761 $15 $91 $150 $25 $28 $101 $20 $94 $159 $138 $290 $0 $30 $0 $160 $129 $78 $122 $31 $50 $270 $4 $46 $248 $10 $160 $105 $5 $9 $154 $22 $1360 $4 $96 $230 $41 $205 $25 $39 $17 $56 $1 $290 $2 $1752 $10 $5 $5 $10 $10 $170 $12 $23 $31 $7 $32 $26 $48 $54 $35 $300 $26 $250 $8 $43 $40 $20 $19 $0 $24 $15 $340 $15 $0 $9 $70 $26 $1 